In [1]:
import os
import re
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [4]:
#Достаем правильно разбитые предложения:

with open ('gold.txt', 'r', encoding = 'utf-8') as g:
    gold_sent = g.read()
gold = re.split(r'\n', gold_sent)

gold[:15]

['ПРЕДИСЛОВИЕ',
 'Наше время отличается не только стремительно растущими возможностями человека, но и лавиноподобным нарастанием проблем, угрожающе неотвратимых и неразрешимых.',
 'Среди них — наводящее страх удушье от экологической петли, неудержимо сжимающейся на шее человечества, тающие на наших глазах запасы земных ресурсов, устрашающий размах планетарной эпидемии СПИДа, необузданный разгул изощренного человеческого эгоизма от половой вседозволенности до расовой ненависти.',
 'Усиливающаяся напряженность человеческого бытия побуждает многих искать выхода из тупиковой ситуации, вызывает в них потребность в утешении, светлой надежде, тихой и спокойной пристани. ',
 'Именно теперь, когда каждый из нас особенно ясно ощущает хрупкость и беззащитность как личного, так и общечеловеческого существования, мы более чем когда-либо нуждаемся в непоколебимом основании непреходящих духовных ценностей, немеркнущих нравственных идеалов и светлых перспектив вечности.',
 'Всякий ищущий обретает эту 

In [27]:
#Достаем кусок книги и разбиваем на предл-я через рег.выр.

regex = r'(?<=[\.\?!] (?=[А-ЯЁA-Z]))'

with open ('piece.txt', 'r', encoding = 'utf-8') as p:
    piece = p.read()
    
piece = re.sub(r'\n', ' ', piece)
piece = re.sub(r'  ', ' ', piece)

reg_sent = re.split(regex, piece)

reg_sent[:15]

['ПРЕДИСЛОВИЕ Наше время отличается не только стремительно растущими возможностями человека, но и лавиноподобным нарастанием проблем, угрожающе неотвратимых и неразрешимых. ',
 'Среди них — наводящее страх удушье от экологической петли, неудержимо сжимающейся на шее человечества, тающие на наших глазах запасы земных ресурсов, устрашающий размах планетарной эпидемии СПИДа, необузданный разгул изощренного человеческого эгоизма от половой вседозволенности до расовой ненависти. ',
 'Усиливающаяся напряженность человеческого бытия побуждает многих искать выхода из тупиковой ситуации, вызывает в них потребность в утешении, светлой надежде, тихой и спокойной пристани. ',
 'Именно теперь, когда каждый из нас особенно ясно ощущает хрупкость и беззащитность как личного, так и общечеловеческого существования, мы более чем когда-либо нуждаемся в непоколебимом основании непреходящих духовных ценностей, немеркнущих нравственных идеалов и светлых перспектив вечности. ',
 'Всякий ищущий обретает эту ж

In [28]:
# Сравниваем кол-во предл-й, полученных с помощью рег.выр., с кол-вом разбитых вручную

print('Gold length - ', len(gold))
print('My split - ', len(reg_sent))

Gold length -  51
My split -  53


In [29]:
# пересечение множеств

len(set(gold) & set(reg_sent)) / len(set(gold) | set(reg_sent))

0.31645569620253167

In [30]:
# открываем весь текст книги и тренируем модель

with open ('text.txt', 'r', encoding = 'utf-8') as t:
    text = t.read()
text = re.sub(r'\n', ' ', text)
text = re.sub(r'  ', ' ', text)

In [31]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [39]:
# получаем предложения с помощью модели и далее оцениваем точность

tokenized = tokenizer.tokenize(piece)

tokenized

['ПРЕДИСЛОВИЕ Наше время отличается не только стремительно растущими возможностями человека, но и лавиноподобным нарастанием проблем, угрожающе неотвратимых и неразрешимых.',
 'Среди них — наводящее страх удушье от экологической петли, неудержимо сжимающейся на шее человечества, тающие на наших глазах запасы земных ресурсов, устрашающий размах планетарной эпидемии СПИДа, необузданный разгул изощренного человеческого эгоизма от половой вседозволенности до расовой ненависти.',
 'Усиливающаяся напряженность человеческого бытия побуждает многих искать выхода из тупиковой ситуации, вызывает в них потребность в утешении, светлой надежде, тихой и спокойной пристани.',
 'Именно теперь, когда каждый из нас особенно ясно ощущает хрупкость и беззащитность как личного, так и общечеловеческого существования, мы более чем когда-либо нуждаемся в непоколебимом основании непреходящих духовных ценностей, немеркнущих нравственных идеалов и светлых перспектив вечности.',
 'Всякий ищущий обретает эту жизне

In [35]:
print('Gold length - ', len(gold))
print('Tokenized - ', len(tokenized))

Gold length -  51
Tokenized -  54


In [38]:
tp = 0
fp = 0
fn = 0

for sent in gold:
    if len(tokenizer.tokenize(sent)) == 1:
        tp += 1
    else:
        fp += 1

for i in range(len(gold)-1):
    sent1, sent2 = gold[i], gold[i+1]
    sent = ' '.join([sent1, sent2])
    if len(tokenizer.tokenize(sent)) == 2:
        tp += 1
    else:
        fn += 1

precision = (tp/(tp+fp))
recall = (tp/(tp+fn))
f1 = 2*(precision*recall)/(precision+recall)
print('Precision - ', precision)
print('Recall - ', recall)
print('F1 - ', f1)

Precision -  0.945054945054945
Recall -  0.8958333333333334
F1 -  0.9197860962566845
